# pyfredapi

A Python library for the Federal Reserve Economic Data (FRED) API: 
https://fred.stlouisfed.org/docs/api/fred/

## Github

https://github.com/gw-moore/pyfredapi

## Documentation

https://pyfredapi.readthedocs.io/en/latest/

## Description

pyfredapi is a Python library that makes it is easy to retrieve data from the FRED API web service.

pyfredapi covers all the FRED API endpoints, and can retrieve data from FRED and ALFRED. Data can be returned as a pandas dataframe or as json. Requests to the FRED API can be customized according to the parameters made available by the web service endpoints.

In [7]:
# Install dependencies
%pip install python-dotenv


[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# FRED API key
FRED_API_KEY = "your_fred_api_key_here"

if not os.environ.get("FRED_API_KEY"):
    os.environ["FRED_API_KEY"] = FRED_API_KEY

In [9]:
# Install packages
%pip install pyfredapi rprint


[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from rich import print as rprint
from rich.pretty import pprint

import pyfredapi as pf

## Macroeconomic Parameters: 

### - GDP (https://fred.stlouisfed.org/series/GDP)
### - Federal Funds Effective Rate (https://fred.stlouisfed.org/series/DFF)
### - Unemployment rate (https://fred.stlouisfed.org/series/UNRATE)

## Series Metadata

Before looking at series data, it is helpful to understand the pyfredapi's get_series_info function and SeriesInfo object. Each series published by FRED has associated metadata such as:

- Start and end dates
- Publish frequency
- Unit of measure

You can query a series' information directly with get_series_info. The get_series_info function returns a SeriesInfo object that contains all the metadata for the given series.

In the below example, we request information for the U.S. GDP series. From the result, we can see that the GDP series is

- Published quarterly
- The earliest data available is 1947-01-01
- The unit of measure is Billions of Dollars

In [11]:
from datetime import datetime

# Get the current year
current_year = datetime.now().year

# Get past year
three_years = current_year - 3

# Define the observation start and end dates for the current year
# https://pyfredapi.readthedocs.io/en/latest/tutorials/series/#get-releases-as-of-date
extra_parameters = {
    "observation_start": f"{three_years}-01-01",
    "observation_end": f"{current_year}-12-31",
}

### Gross Domestic Product (GDP)

Source: U.S. Bureau of Economic Analysis  Release: Gross Domestic Product  
Units:  Billions of Dollars, Seasonally Adjusted Annual Rate

Frequency:  Quarterly

Notes:
BEA Account Code: A191RC

Gross domestic product (GDP), the featured measure of U.S. output, is the market value of the goods and services produced by labor and property located in the United States.For more information, see the Guide to the National Income and Product Accounts of the United States (NIPA) and the Bureau of Economic Analysis.

Suggested Citation:
U.S. Bureau of Economic Analysis, Gross Domestic Product [GDP], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/GDP, March 4, 2025.

### U.S. GDP series

In [12]:
gdp_info = pf.get_series_info(series_id="GDP", api_key=FRED_API_KEY)

# Using rich to pretty print the SeriesInfo
rprint(gdp_info)

SeriesInfo(
    id='GDP',
    realtime_start='2025-03-27',
    realtime_end='2025-03-27',
    title='Gross Domestic Product',
    observation_start='1947-01-01',
    observation_end='2024-10-01',
    frequency='Quarterly',
    frequency_short='Q',
    units='Billions of Dollars',
    units_short='Bil. of $',
    seasonal_adjustment='Seasonally Adjusted Annual Rate',
    seasonal_adjustment_short='SAAR',
    last_updated='2025-03-27 08:03:26-05',
    popularity=93,
    notes='BEA Account Code: A191RC\n\nGross domestic product (GDP), the featured measure of U.S. output, is the 
market value of the goods and services produced by labor and property located in the United States.For more 
information, see the Guide to the National Income and Product Accounts of the United States (NIPA) and the Bureau 
of Economic Analysis (http://www.bea.gov/national/pdf/nipaguid.pdf).'
)

In [14]:
# https://pyfredapi.readthedocs.io/en/latest/tutorials/series/#pandas-dataframe
gdp_df = pf.get_series(series_id="GDP", **extra_parameters, api_key=FRED_API_KEY)

In [15]:
gdp_df.dtypes

realtime_start            object
realtime_end              object
date              datetime64[ns]
value                    float64
dtype: object

In [16]:
gdp_df.head(5)

,realtime_start,realtime_end,date,value
0,2025-04-30,2025-04-30,2022-01-01,25215.491
1,2025-04-30,2025-04-30,2022-04-01,25805.791
2,2025-04-30,2025-04-30,2022-07-01,26272.011
3,2025-04-30,2025-04-30,2022-10-01,26734.277
4,2025-04-30,2025-04-30,2023-01-01,27164.359


In [17]:
gdp_df.tail(5)

,realtime_start,realtime_end,date,value
7,2025-04-30,2025-04-30,2023-10-01,28296.967
8,2025-04-30,2025-04-30,2024-01-01,28624.069
9,2025-04-30,2025-04-30,2024-04-01,29016.714
10,2025-04-30,2025-04-30,2024-07-01,29374.914
11,2025-04-30,2025-04-30,2024-10-01,29723.864


In [18]:
gdp_df.to_csv("gdp_index_historical.csv", index=False)

In [20]:
gdp_json = pf.get_series(series_id="GDP", return_format="json", **extra_parameters, api_key=FRED_API_KEY)

In [21]:
pprint(gdp_json, max_length=10)

[
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-01', 'value': '25215.491'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-04-01', 'value': '25805.791'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-07-01', 'value': '26272.011'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-10-01', 'value': '26734.277'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2023-01-01', 'value': '27164.359'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2023-04-01', 'value': '27453.815'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2023-07-01', 'value': '27967.697'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2023-10-01', 'value': '28296.967'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2024-01-01', 'value': '28624.069'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2024-04-01', 'value': '29016.714'},
│   ... +2
]

### Effective Interest Rate

### Federal Funds Effective Rate (DFF)

Observations 

2025-04-28: 4.33
Updated: Apr 29, 2025 3:16 PM CDT
Next Release Date: Apr 30, 2025
Units:

Percent,
Not Seasonally Adjusted
Frequency:

Daily,
7-Day

Notes
Source: Board of Governors of the Federal Reserve System (US)  Release: H.15 Selected Interest Rates  
Units:  Percent, Not Seasonally Adjusted

Frequency:  Daily, 7-Day

Notes:
For additional historical federal funds rate data, please see Daily Federal Funds Rate from 1928-1954.

The federal funds rate is the interest rate at which depository institutions trade federal funds (balances held at Federal Reserve Banks) with each other overnight. When a depository institution has surplus balances in its reserve account, it lends to other banks in need of larger balances. In simpler terms, a bank with excess cash, which is often referred to as liquidity, will lend to another bank that needs to quickly raise liquidity. (1) The rate that the borrowing institution pays to the lending institution is determined between the two banks; the weighted average rate for all of these types of negotiations is called the effective federal funds rate.(2) The effective federal funds rate is essentially determined by the market but is influenced by the Federal Reserve through open market operations to reach the federal funds rate target.(2)

The Federal Open Market Committee (FOMC) meets eight times a year to determine the federal funds target rate. As previously stated, this rate influences the effective federal funds rate through open market operations or by buying and selling of government bonds (government debt).(2) More specifically, the Federal Reserve decreases liquidity by selling government bonds, thereby raising the federal funds rate because banks have less liquidity to trade with other banks. Similarly, the Federal Reserve can increase liquidity by buying government bonds, decreasing the federal funds rate because banks have excess liquidity for trade. Whether the Federal Reserve wants to buy or sell bonds depends on the state of the economy. If the FOMC believes the economy is growing too fast and inflation pressures are inconsistent with the dual mandate of the Federal Reserve, the Committee may set a higher federal funds rate target to temper economic activity. In the opposing scenario, the FOMC may set a lower federal funds rate target to spur greater economic activity. Therefore, the FOMC must observe the current state of the economy to determine the best course of monetary policy that will maximize economic growth while adhering to the dual mandate set forth by Congress. In making its monetary policy decisions, the FOMC considers a wealth of economic data, such as: trends in prices and wages, employment, consumer spending and income, business investments, and foreign exchange markets.

The federal funds rate is the central interest rate in the U.S. financial market. It influences other interest rates such as the prime rate, which is the rate banks charge their customers with higher credit ratings. Additionally, the federal funds rate indirectly influences longer- term interest rates such as mortgages, loans, and savings, all of which are very important to consumer wealth and confidence.(2)

In [22]:
effective_interest_rate_info = pf.get_series_info(series_id="DFF", api_key=FRED_API_KEY)
# Using rich to pretty print the SeriesInfo
rprint(effective_interest_rate_info)

SeriesInfo(
    id='DFF',
    realtime_start='2025-04-30',
    realtime_end='2025-04-30',
    title='Federal Funds Effective Rate',
    observation_start='1954-07-01',
    observation_end='2025-04-28',
    frequency='Daily, 7-Day',
    frequency_short='D',
    units='Percent',
    units_short='%',
    seasonal_adjustment='Not Seasonally Adjusted',
    seasonal_adjustment_short='NSA',
    last_updated='2025-04-29 15:16:40-05',
    popularity=86,
    notes=' Daily Federal Funds Rate from 1928-1954 (https://fred.stlouisfed.org/categories/33951).\n\nThe federal 
funds rate is the interest rate at which depository institutions trade federal funds (balances held at Federal 
Reserve Banks) with each other overnight. When a depository institution has surplus balances in its reserve 
account, it lends to other banks in need of larger balances. In simpler terms, a bank with excess cash, which is 
often referred to as liquidity, will lend to another bank that needs to quickly raise liquidity. (1) The rate that 
the borrowing institution pays to the lending institution is determined between the two banks; the weighted average
rate for all of these types of negotiations is called the effective federal funds rate.(2) The effective federal 
funds rate is essentially determined by the market but is influenced by the Federal Reserve through open market 
operations to reach the federal funds rate target.(2)\n\nThe Federal Open Market Committee (FOMC) meets eight times
a year to determine the federal funds target rate. As previously stated, this rate influences the effective federal
funds rate through open market operations or by buying and selling of government bonds (government debt).(2) More 
specifically, the Federal Reserve decreases liquidity by selling government bonds, thereby raising the federal 
funds rate because banks have less liquidity to trade with other banks. Similarly, the Federal Reserve can increase
liquidity by buying government bonds, decreasing the federal funds rate because banks have excess liquidity for 
trade. Whether the Federal Reserve wants to buy or sell bonds depends on the state of the economy. If the FOMC 
believes the economy is growing too fast and inflation pressures are inconsistent with the dual mandate of the 
Federal Reserve, the Committee may set a higher federal funds rate target to temper economic activity. In the 
opposing scenario, the FOMC may set a lower federal funds rate target to spur greater economic activity. Therefore,
the FOMC must observe the current state of the economy to determine the best course of monetary policy that will 
maximize economic growth while adhering to the dual mandate set forth by Congress. In making its monetary policy 
decisions, the FOMC considers a wealth of economic data, such as: trends in prices and wages, employment, consumer 
spending and income, business investments, and foreign exchange markets.\n\nThe federal funds rate is the central 
interest rate in the U.S. financial market. It influences other interest rates such as the prime rate, which is the
rate banks charge their customers with higher credit ratings. Additionally, the federal funds rate indirectly 
influences longer- term interest rates such as mortgages, loans, and savings, all of which are very important to 
consumer wealth and confidence.(2)\n\nReferences\n(1) Federal Reserve Bank of New York. "Federal funds." Fedpoints,
August 2007.\n(2) Monetary Policy (https://www.federalreserve.gov/monetarypolicy.htm), Board of Governors of the 
Federal Reserve System.\n\nFor questions on the data, please contact the data source 
(https://www.federalreserve.gov/apps/ContactUs/feedback.aspx?refurl=/releases/h15/%). For questions on FRED 
functionality, please contact us here (https://fred.stlouisfed.org/contactus/).</p>'
)

In [24]:
effective_interest_rate_df = pf.get_series(series_id="DFF", **extra_parameters, api_key=FRED_API_KEY)

In [25]:
effective_interest_rate_df.dtypes

realtime_start            object
realtime_end              object
date              datetime64[ns]
value                    float64
dtype: object

In [26]:
effective_interest_rate_df.tail(5)

,realtime_start,realtime_end,date,value
1209,2025-04-30,2025-04-30,2025-04-24,4.33
1210,2025-04-30,2025-04-30,2025-04-25,4.33
1211,2025-04-30,2025-04-30,2025-04-26,4.33
1212,2025-04-30,2025-04-30,2025-04-27,4.33
1213,2025-04-30,2025-04-30,2025-04-28,4.33


In [27]:
effective_interest_rate_df.head(5)

,realtime_start,realtime_end,date,value
0,2025-04-30,2025-04-30,2022-01-01,0.07
1,2025-04-30,2025-04-30,2022-01-02,0.07
2,2025-04-30,2025-04-30,2022-01-03,0.08
3,2025-04-30,2025-04-30,2022-01-04,0.08
4,2025-04-30,2025-04-30,2022-01-05,0.08


In [28]:
effective_interest_rate_df.to_csv("effective_interest_rate_historical.csv", index=False)

In [30]:
effective_interest_rate_json = pf.get_series(series_id="DFF", return_format="json", **extra_parameters, api_key=FRED_API_KEY)
pprint(effective_interest_rate_json, max_length=20)

[
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-01', 'value': '0.07'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-02', 'value': '0.07'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-03', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-04', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-05', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-06', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-07', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-08', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-09', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-10', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-11', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-12', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-13', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-14', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-15', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-16', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-17', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-18', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-19', 'value': '0.08'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-20', 'value': '0.08'},
│   ... +1194
]

### U.S. Unemployment Rate

https://fred.stlouisfed.org/series/UNRATE

Source: U.S. Bureau of Labor Statistics  Release: Employment Situation  
Units:  Percent, Seasonally Adjusted

Frequency:  Monthly

Notes:
The unemployment rate represents the number of unemployed as a percentage of the labor force. Labor force data are restricted to people 16 years of age and older, who currently reside in 1 of the 50 states or the District of Columbia, who do not reside in institutions (e.g., penal and mental facilities, homes for the aged), and who are not on active duty in the Armed Forces.

This rate is also defined as the U-3 measure of labor underutilization.

The series comes from the 'Current Population Survey (Household Survey)'

The source code is: LNS14000000

Suggested Citation:
U.S. Bureau of Labor Statistics, Unemployment Rate [UNRATE], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/UNRATE, February 14, 2025.

In [32]:
unrate_info = pf.get_series_info(series_id="UNRATE", api_key=FRED_API_KEY)

# Using rich to pretty print the SeriesInfo
rprint(unrate_info)

SeriesInfo(
    id='UNRATE',
    realtime_start='2025-04-04',
    realtime_end='2025-04-04',
    title='Unemployment Rate',
    observation_start='1948-01-01',
    observation_end='2025-03-01',
    frequency='Monthly',
    frequency_short='M',
    units='Percent',
    units_short='%',
    seasonal_adjustment='Seasonally Adjusted',
    seasonal_adjustment_short='SA',
    last_updated='2025-04-04 07:48:18-05',
    popularity=95,
    notes="The unemployment rate represents the number of unemployed as a percentage of the labor force. Labor 
force data are restricted to people 16 years of age and older, who currently reside in 1 of the 50 states or the 
District of Columbia, who do not reside in institutions (e.g., penal and mental facilities, homes for the aged), 
and who are not on active duty in the Armed Forces.\r\n\r\nThis rate is also defined as the U-3 measure of labor 
underutilization.\r\n\r\nThe series comes from the 'Current Population Survey (Household Survey)'\r\n\r\nThe source
code is: LNS14000000"
)

### UNRATE series

In [34]:
# Fetch the UNRATE series data
unrate_df = pf.get_series(series_id="UNRATE", **extra_parameters, api_key=FRED_API_KEY)

In [35]:
unrate_df.dtypes

realtime_start            object
realtime_end              object
date              datetime64[ns]
value                    float64
dtype: object

In [36]:
unrate_df.head(5)

,realtime_start,realtime_end,date,value
0,2025-04-30,2025-04-30,2022-01-01,4.0
1,2025-04-30,2025-04-30,2022-02-01,3.8
2,2025-04-30,2025-04-30,2022-03-01,3.7
3,2025-04-30,2025-04-30,2022-04-01,3.7
4,2025-04-30,2025-04-30,2022-05-01,3.6


In [37]:
unrate_df.tail(5)

,realtime_start,realtime_end,date,value
34,2025-04-30,2025-04-30,2024-11-01,4.2
35,2025-04-30,2025-04-30,2024-12-01,4.1
36,2025-04-30,2025-04-30,2025-01-01,4.0
37,2025-04-30,2025-04-30,2025-02-01,4.1
38,2025-04-30,2025-04-30,2025-03-01,4.2


In [38]:
unrate_df.to_csv("unemployment_rate_historical.csv", index=False)

In [40]:
# Fetch the UNRATE series data
unrate_json = pf.get_series(series_id="UNRATE", return_format="json", **extra_parameters, api_key=FRED_API_KEY)
pprint(unrate_json, max_length=20)

[
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-01-01', 'value': '4.0'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-02-01', 'value': '3.8'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-03-01', 'value': '3.7'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-04-01', 'value': '3.7'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-05-01', 'value': '3.6'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-06-01', 'value': '3.6'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-07-01', 'value': '3.5'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-08-01', 'value': '3.6'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-09-01', 'value': '3.5'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-10-01', 'value': '3.6'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-11-01', 'value': '3.6'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2022-12-01', 'value': '3.5'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2023-01-01', 'value': '3.5'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2023-02-01', 'value': '3.6'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2023-03-01', 'value': '3.5'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2023-04-01', 'value': '3.4'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2023-05-01', 'value': '3.6'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2023-06-01', 'value': '3.6'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2023-07-01', 'value': '3.5'},
│   {'realtime_start': '2025-04-30', 'realtime_end': '2025-04-30', 'date': '2023-08-01', 'value': '3.7'},
│   ... +19
]